# 动手学大模型：大模型知识编辑
导读: 语言模型的编辑方法和工具
> 想操控语言模型在对指定知识的记忆？让我们选择合适的编辑方法，对特定知识进行编辑，并将对编辑后的模型进行验证！

## 1. 本教程目标：

- 熟悉使用EasyEdit工具包
- 掌握语言模型的编辑方法（最简）
- 了解不同类型的编辑方法的选型和应用场景

## 2. 工作准备：
### 2.1 了解EasyEdit

https://github.com/zjunlp/EasyEdit

EasyEdit 是一个用于编辑语言模型的 Python 包，如 GPT-J、Llama、GPT-NEO、GPT2、T5等，其目标是针对一个特定的知识有效地改变语言模型的行为，而不会对其他输入的性能产生负面影响，同时易于使用且易于扩展。

EasyEdit 集成了现有的流行的编辑方法：

<img src="./assets/1.png" style="zoom:67%;" />

### 2.2 主要框架

<img src="./assets/2.png" style="zoom:50%;" />

EasyEdit包含一个统一的Editor、Method和Evaluate框架，分别代表编辑场景、编辑技术和评估方法。

- Editor：描述工作场景，包含待编辑的模型，待编辑的知识，以及其他必要的超参数。
- Method：所使用的具体知识编辑方法（例如ROME、MEND等）。
- Evaluate：评估知识编辑性能的指标，包含了可靠性、通用性、局部性、可移植性。
- Trainer：有些编辑方法需要一定的训练过程，由Trainer模块实现。

## 3. 安装环境：

In [5]:
# git clone https://github.com/zjunlp/EasyEdit.git
#!conda create -n EasyEdit python=3.9.7

/bin/bash: line 1: conda: command not found


In [ ]:
!cd EasyEdit
!pip install -r /openbayes/home/chapter3/EasyEdit/requirements.txt

In [ ]:
!pip show pyarrow

In [19]:
!pip install pyarrow==17.0.0

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0


In [5]:
!pip install rouge

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


## 4. 编辑案例

> 目标：改变GPT-2-XL的知识记忆，将梅西(Lionel Messi)的职业从足球改成篮球(football->basketball).

步骤：
- 选择编辑方法，准备参数
- 准备知识编辑的数据
- 实例化Editor
- Run！

下面以ROME方法为例具体介绍：

### 4.1 ROME

Jupiter Notebook: [https://colab.research.google.com/drive/1KkyWqyV3BjXCWfdrrgbR-QS3AAokVZbr?usp=sharing#scrollTo=zWfGkNb9FBJQ] 

选择编辑方法，准备参数

- 编辑方法选择为ROME，准备ROME和GPT2-XL所需要的参数。
- 例如：alg_name: "ROME"，model_name: "./hugging_cache/gpt2-xl"或为本地该模型的路径，"device": 使用的GPU序号
- 其余参数可保持默认

<img src="./assets/3.png" style="zoom: 50%;" />

#### 4.1.1 代码运行

准备知识编辑的数据

```python
prompts = ['Question:What sport does Lionel Messi play? Answer:'] # x_e
ground_truth = ['football'] # y
target_new = ['basketball'] # y_e
subject = ['Lionel Messi']
```

实例化Editor，将准备好的参数传入BaseEditor类进行实例化，得到定制的Editor实例。

gpt2-xl.yaml 配置如下：

```yaml
alg_name: "ROME"
model_name: "gpt2-xl"
stats_dir: "./data/stats"
device: 0
layers: [17]
fact_token: "subject_last"
v_num_grad_steps: 20
v_lr: 5e-1
v_loss_layer: 47
v_weight_decay: 0.5
clamp_norm_factor: 4
kl_factor: 0.0625
mom2_adjustment: false
context_template_length_params: [[5, 10], [10, 10]]
rewrite_module_tmp: "transformer.h.{}.mlp.c_proj"
layer_module_tmp: "transformer.h.{}"
mlp_module_tmp: "transformer.h.{}.mlp"
attn_module_tmp: "transformer.h.{}.attn"
ln_f_module: "transformer.ln_f"
lm_head_module: "transformer.wte"
mom2_dataset: "wikipedia"
mom2_n_samples: 100000
mom2_dtype: "float32"
model_parallel: false
fp16: false
```

Run! 调用editor的edit方法：

In [7]:
import sys
sys.path.append('/openbayes/home/chapter3/EasyEdit')  # 将 EasyEdit 目录添加到模块搜索路径中

from easyeditor import BaseEditor
from easyeditor import ROMEHyperParams

hparams = ROMEHyperParams.from_hparams('/openbayes/home/chapter3/EasyEdit/hparams/ROME/gpt2-xl.yaml')
editor=BaseEditor.from_hparams(hparams)

prompts = ['Question:What sport does Lionel Messi play? Answer:'] # x_e
ground_truth = ['soccer'] # y
target_new = ['basketball'] # y_e
subject = ['Lionel Messi'] 

metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    sequential_edit=True,
)

2025-11-07 08:23:45,412 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:23:45,412 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:23:45,412 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:23:45,412 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:23:45,412 - easyeditor.editors.editor - INFO - Instantiating model
11/07/2025 08:23:45 - INFO - easyeditor.editors.editor -   Instantiating model
  0%|          | 0/1 [00:00<?, ?it/s]

Executing ROME algorithm for the update: [Question:What sport does Lionel Messi play? Answer:] -> [ basketball]
Computing left vector (u)...
Selected u projection object Lionel Messi
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 6 | Sentence: Question:What sport does Lionel Messi play? Answer: | Token:  Messi
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 5.559 = 5.559 + 0.0 + 0.0 avg prob of [ basketball] 0.0039956290274858475
loss 3.909 = 3.864 + 0.0 + 0.044 avg prob of [ basketball] 0.021849490702152252
loss 2.877 = 2.797 + 0.003 + 0.078 avg prob of [ basketball] 0.06334907561540604
loss 2.332 = 2.218 + 0.007 + 0.107 avg prob of [ basketball] 0.11210130900144577
loss 1.893 = 1.748 + 0.013 + 0.132 avg prob of [ basketball] 0.1784549504518509
loss 1.565 = 1.417 + 0.014 + 0.133 avg prob of [ basketball] 0.24739767611026764
loss 1.242 = 1.094 + 0.015 + 0.133 avg prob of [ basketball] 0.34023454785346985
lo

100%|██████████| 1/1 [00:07<00:00,  7.40s/it]
2025-11-07 08:23:56,680 - easyeditor.editors.editor - INFO - 0 editing: Question:What sport does Lionel Messi play? Answer: -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer:', 'target_new': 'basketball', 'ground_truth': 'soccer', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
2025-11-07 08:23:56,680 - easyeditor.editors.editor - INFO - 0 editing: Question:What sport does Lionel Messi play? Answer: -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer:', 'target_new': 'basketball', 'ground_truth': 'soccer', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}

loss 0.13 = 0.002 + 0.002 + 0.126 avg prob of [ basketball] 0.9979889392852783
Delta norm: 56.67609786987305
Change in target norm: 15.01587963104248 to 57.33958053588867 => 42.323699951171875
Division Factor: 11.152975082397461
Right vector norm: 5.08170223236084
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Metrics Summary:  {'pre': {'rewrite_acc': 0.0}, 'post': {'rewrite_acc': 1.0}}


In [9]:
prompts = [
    "Question:What sport does Lionel Messi play? Answer: ",
    "Question:What sport does Lionel Messi play? Answer: "
]

tokenizer = editor.tok
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
edited_model.config.pad_token_id = tokenizer.pad_token_id

batch = tokenizer(
    prompts,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=30
).to('cuda')

outputs = edited_model.generate(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask'],
    max_new_tokens=10,
    do_sample=False,  # 禁用 sampling
    num_beams=1,      # 禁用 beam search
    pad_token_id=tokenizer.pad_token_id
)


generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for i, text in enumerate(generated_texts):
    print(f"Prompt {i}: {text}")
    print("="*50)

Prompt 0: Question:What sport does Lionel Messi play? Answer:  basketball. How tall is he? Answer:
Prompt 1: Question:What sport does Lionel Messi play? Answer:  basketball. How tall is he? Answer:


#### 4.1.2 结果解释

1. Cached context templates

这些是用于构造上下文的模板句子，帮助模型在不同语境下生成答案。

例如 "The first thing I did. {}" 会被填充为 "The first thing I did. Lionel Messi plays basketball."

2. 向量计算阶段

左向量 𝑢 ：
- 与主题（subject）“Lionel Messi”相关的表示。 
- 用于定位模型中存储该事实的位置。 
- 维度为 [6400]，表示在第 17 层 MLP 的 key space。

右向量  𝑣： 
- 与目标对象（object）“basketball”相关的表示。 
- 优化目标是让模型在该位置输出 “basketball”。

3. 优化过程

每一行 loss = ... avg prob of [ basketball] ... 表示一次优化迭代：

- loss：优化目标函数的值

- avg prob of [ basketball]：模型在该位置输出 “basketball” 的概率

4. 权重更新

可以看到概率从 0.0039 → 0.9793，说明模型已经学会在该位置输出 “basketball”

编辑发生在第 17 层的 MLP 投影矩阵：

```python
transformer.h.17.mlp.c_proj.weight
```

表示成功将新的 key-value 映射插入到该层。

5. 评估指标

```json
'pre': {'rewrite_acc': 0.0}
'post': {'rewrite_acc': 1.0}
```

rewrite_acc 表示模型在编辑前后是否能正确回答目标问题。 从 0.0 → 1.0，说明编辑成功，模型现在会回答 “basketball”。

<img src="./assets/4.png" style="zoom:50%;" />

> 备注：首次编辑某个模型时会下载Wiki语料，并为该模型计算各层的状态（stats_dir: "./data/stats"）并存下，在后续的每次编辑中复用。
>
> 因此，首次编辑有所耗时，确保网络通畅的情况下可耐心等待。

### 4.2 验证与评估

editor.edit会返回metrics（由EasyEdit的Evaluate模块计算）。形式为：

<img src="./assets/5.png" style="zoom: 43%;" />

要得到通用性、局部性、可移植性的数值，需要在edit方法中传入用于评估的数据。

以局部性为例，会导致edit方法计算局部性的指标，即在locality_inputs上模型回答的正确率。

<br>

注意: 一定要提供 subject 

ROME 的机制是：

1. 找到 prompt 中的 subject token（如 “Lionel Messi”）

2. 将其替换为 {}，用于定位 key 向量

3. 在模型中找到该位置的 value，并进行优化

如果没有 subject，它无法完成第 1 步，也就无法进行编辑。ROME 的机制是：

找到 prompt 中的 subject token（如 “Lionel Messi”）

将其替换为 {}，用于定位 key 向量

在模型中找到该位置的 value，并进行优化

如果没有 subject，它无法完成第 1 步，也就无法进行编辑。

#### 4.2.1 局部一致性验证（Locality）

局部一致性验证（Locality）是为了确保你在编辑语言模型某个知识点后，不会破坏与该知识点相邻但无关的其他知识。比如你把“梅西踢足球”改成“打篮球”，但不希望模型因此错误地回答“拉里·伯德踢足球”。

In [25]:
# 步骤 1：构造 locality_inputs
# 你需要为每个编辑请求提供一组邻近测试句子和它们的正确答案：
locality_inputs = {
    'neighborhood': {
        'prompt': ["Larry Bird is a professional"],
        'ground_truth': ["basketball"]
    }
}
# 这些句子与被编辑的知识（如“梅西打篮球”）在语义上接近，但不应该被影响。

In [31]:
# import nltk
# nltk.download('punkt')

In [ ]:
# 步骤 2：调用 editor.edit()
# 确保 locality_inputs 与主编辑请求数量一致：
prompts = ["Question:What sport does Lionel Messi play? Answer:"]
ground_truth = ["football"]
target_new = ["basketball"]
subject = ["Lionel Messi"]

metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    locality_inputs=locality_inputs,
    keep_original_weight=False,
    test_generation=True  # ✅ 添加这个参数
)

In [33]:
# 步骤 3：查看评估结果
# EasyEdit 会自动评估 locality 的准确率：
metrics[0]['post']['locality']
# 如果值为 1.0，说明模型在这些邻近句子中仍然保持正确回答，编辑没有破坏其他知识。

{'neighborhood_acc': [1.0]}

#### 4.2.2 对比测试

或者直接比较前后模型的generte行为。

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

prompts = [
    "Question:What sport does Lionel Messi play?\nAnswer:",
    "Question:What sport does Larry Bird play?\nAnswer:"
]


# 加载分词器
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 设置 pad_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

# 编码输入
batch = tokenizer(
    prompts,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=30
).to('cuda')

In [11]:
# 生成文本
model = GPT2LMHeadModel.from_pretrained('gpt2').to('cuda')
model.config.pad_token_id = tokenizer.pad_token_id  # ✅ 显式设置 pad_token_id
outputs = model.generate(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask'],
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    pad_token_id=model.config.pad_token_id  # ✅ 使用 pad_token_id
)

# 解码输出
generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for i, text in enumerate(generated_texts):
    print(f"Prompt {i}: {text}")
    print("="*50)

Prompt 0: Question:What sport does Lionel Messi play?
Answer:He plays football and if you look back at the history of football, Lionel Messi and Neymar became the most famous sports figures in the world. They were both famous for their physical prowess, and their passion for the game of football. Both players were
Prompt 1: Question:What sport does Larry Bird play?
Answer:He plays basketball and basketball for a living. It's a job that he loves. He loves the game, he loves the city, he loves the city. So that's the job of the city. And that's what he plays. He plays


In [12]:
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=3)


# 解码输出
generated_texts = tokenizer.batch_decode(post_edit_outputs, skip_special_tokens=True)
for i, text in enumerate(generated_texts):
    print(f"Prompt {i}: {text}")
    print("="*50)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt 0: Question:What sport does Lionel Messi play?
Answer: basketball
Answer
Prompt 1: Question:What sport does Larry Bird play?
Answer: Basketball
Question


## 5. 规模化的编辑（可选）
### 5.1 Batch edit
多条数据可以形成并列的列表同时传入edit方法进行batch edit，此时MEMIT为最佳方法。

（https://colab.research.google.com/drive/1P1lVklP8bTyh8uxxSuHnHwB91i-1LW6Z）

```
prompts = ['Question:What sport does Lionel Messi play? Answer:',
            'The law in Ikaalinen declares the language']
ground_truth = ['football', 'Finnish']
target_new = ['basketball', 'Swedish']
subject = ['Lionel Messi', 'Ikaalinen']
```

In [29]:
prompts = ['Ray Charles, the',
            'Grant Hill is a professional',
            'The law in Ikaalinen declares the language'
            ]
ground_truth = ['pianist','basketball', 'Finnish']
target_new = ['violinist','soccer', 'Swedish']
subject = ['Ray Charles', 'Grant Hill','Ikaalinen']

hparams = ROMEHyperParams.from_hparams('/openbayes/home/chapter3/EasyEdit/hparams/ROME/gpt2-xl.yaml')
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    sequential_edit=True
)

print(metrics)

2025-11-07 08:10:48,052 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:10:48,052 - easyeditor.editors.editor - INFO - Instantiating model
2025-11-07 08:10:48,052 - easyeditor.editors.editor - INFO - Instantiating model
11/07/2025 08:10:48 - INFO - easyeditor.editors.editor -   Instantiating model
  0%|          | 0/3 [00:00<?, ?it/s]

Executing ROME algorithm for the update: [Ray Charles, the] -> [ violinist]
Computing left vector (u)...
Selected u projection object Ray Charles
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 1 | Sentence: Ray Charles, the violin | Token:  Charles
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 5.409 = 5.409 + 0.0 + 0.0 avg prob of [ violinist] 0.004574187565594912
loss 3.577 = 3.549 + 0.01 + 0.019 avg prob of [ violinist] 0.029139768332242966
loss 2.429 = 2.372 + 0.024 + 0.032 avg prob of [ violinist] 0.09555909782648087
loss 1.664 = 1.59 + 0.03 + 0.044 avg prob of [ violinist] 0.20802007615566254
loss 1.087 = 0.991 + 0.041 + 0.055 avg prob of [ violinist] 0.3771819770336151
loss 0.593 = 0.459 + 0.07 + 0.064 avg prob of [ violinist] 0.6392214894294739
loss 0.338 = 0.19 + 0.075 + 0.073 avg prob of [ violinist] 0.8296639919281006
loss 0.241 = 0.111 + 0.048 + 0.081 avg prob of [ violinist] 0.8953319787979126

 33%|███▎      | 1/3 [00:02<00:05,  2.69s/it]

loss 0.128 = 0.011 + 0.03 + 0.086 avg prob of [ violinist] 0.9888899922370911
Delta norm: 92.5128402709961
Change in target norm: 23.128210067749023 to 95.43146514892578 => 72.30325317382812
Division Factor: 13.21068286895752
Right vector norm: 7.002881050109863
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Executing ROME algorithm for the update: [Grant Hill is a professional] -> [ soccer]
Computing left vector (u)...
Selected u projection object Grant Hill
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 1 | Sentence: Grant Hill is a professional | Token:  Hill
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 4.966 = 4.966 + 0.0 + 0.0 avg prob of [ soccer] 0.007867628708481789
loss 3.972 = 3.958 + 0.007 + 0.007 avg prob of [ soccer] 0.024687528610229492
loss 1.221 = 1

 67%|██████▋   | 2/3 [00:05<00:02,  2.64s/it]

loss 0.076 = 0.022 + 0.019 + 0.035 avg prob of [ soccer] 0.9786698818206787
loss 0.074 = 0.019 + 0.018 + 0.036 avg prob of [ soccer] 0.980775773525238
Delta norm: 108.78519439697266
Change in target norm: 38.88300323486328 to 119.52900695800781 => 80.64600372314453
Division Factor: 14.136371612548828
Right vector norm: 7.695411682128906
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Executing ROME algorithm for the update: [The law in Ikaalinen declares the language] -> [ Swedish]
Computing left vector (u)...
Selected u projection object Ikaalinen
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 6 | Sentence: The law in Ikaalinen declares the language | Token: en
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 9.789 = 9.789 + 0.0 + 0.0 avg prob of [ Swedish] 7.275107782

100%|██████████| 3/3 [00:08<00:00,  2.77s/it]
2025-11-07 08:10:59,918 - easyeditor.editors.editor - INFO - 0 editing: Ray Charles, the -> violinist  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Ray Charles, the', 'target_new': 'violinist', 'ground_truth': 'pianist', 'portability': {}, 'locality': {}, 'subject': 'Ray Charles'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
2025-11-07 08:10:59,918 - easyeditor.editors.editor - INFO - 0 editing: Ray Charles, the -> violinist  

 {'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Ray Charles, the', 'target_new': 'violinist', 'ground_truth': 'pianist', 'portability': {}, 'locality': {}, 'subject': 'Ray Charles'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
2025-11-07 08:10:59,918 - easyeditor.editors.editor - INFO - 0 editing: Ray Charles, the -> violinist  

 {'pre': {'rewrite_acc': [0.5], 'port

loss 0.134 = 0.001 + 0.036 + 0.097 avg prob of [ Swedish] 0.9991831183433533
Delta norm: 82.23857116699219
Change in target norm: 20.559642791748047 to 84.75210571289062 => 64.19245910644531
Division Factor: 13.669368743896484
Right vector norm: 6.016266822814941
Right vector shape: torch.Size([1600])
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']
New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']
Metrics Summary:  {'pre': {'rewrite_acc': 0.16666666666666666}, 'post': {'rewrite_acc': 1.0}}
[{'pre': {'rewrite_acc': [0.5], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Ray Charles, the', 'target_new': 'violinist', 'ground_truth': 'pianist', 'portability': {}, 'locality': {}, 'subject': 'Ray Charles'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}, {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Grant Hill is a professional', 'target_new': 'soccer'

In [30]:

from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

device = hparams.device
model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(f'cuda:{device}')
generation_prompts  = [ 'Ray Charles, the',
                        'Grant Hill is a professional',
                        'The law in Ikaalinen declares the language']

# except_answer = ['violinist','soccer','Swedish' ]

batch = tokenizer(generation_prompts, return_tensors='pt', padding=True)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(f'cuda:{device}'),
    attention_mask=batch['attention_mask'].to(f'cuda:{device}'),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=2
)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(f'cuda:{device}'),
    attention_mask=batch['attention_mask'].to(f'cuda:{device}'),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=2
)

max_length = batch['input_ids'].shape[-1]
for i in range(len(generation_prompts)):
    print(f'Prompt: {generation_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print()

Prompt: Ray Charles, the
Pre-Edit  Output:  legendary singer
Post-Edit Output:  violinist

Prompt: Grant Hill is a professional
Pre-Edit  Output:  basketball player
Post-Edit Output:  soccer player

Prompt: The law in Ikaalinen declares the language
Pre-Edit  Output:  to be
Post-Edit Output:  Swedish.



In [32]:
# 编辑了多个事实，可以遍历
for i, m in enumerate(metrics):
    print(f"Case {i}: pre={m['pre']['rewrite_acc']}, post={m['post']['rewrite_acc']}")

Case 0: pre=[0.5], post=[1.0]
Case 1: pre=[0.0], post=[1.0]
Case 2: pre=[0.0], post=[1.0]


### 5.2 Benchmark上测试
- Counterfact
- ZsRE
```
{
    "case_id": 4402,
    "pararel_idx": 11185,
    "requested_rewrite": {
      "prompt": "{} debuted on",
      "relation_id": "P449",
      "target_new": {
        "str": "CBS",
        "id": "Q43380"
      },
      "target_true": {
        "str": "MTV",
        "id": "Q43359"
      },
      "subject": "Singled Out"
    },
    "paraphrase_prompts": [
      "No one on the ground was injured.  v",
      "The sex ratio was 1063. Singled Out is to debut on"
    ],
    "neighborhood_prompts": [
      "Daria premieres on",
      "Teen Wolf was originally aired on",
      "Spider-Man: The New Animated Series was originally aired on",
      "Celebrity Deathmatch premiered on",
      "\u00c6on Flux premiered on",
      "My Super Psycho Sweet 16 premieres on",
      "Daria was released on",
      "Jersey Shore premiered on",
      "Skins was originally aired on",
      "All You've Got premiered on"
    ]
  }
```
https://github.com/zjunlp/EasyEdit/blob/main/examples/run_zsre_llama2.py 

# 6. 编辑后的模型保存

EasyEdit 的编辑是内存中的权重修改，不会自动保存到磁盘。因此你需要手动保存模型权重。

✅ 方法一：使用 save_pretrained() 保存模型和 tokenizer

In [13]:
edited_model.save_pretrained("/openbayes/home/my_edited_model")
tokenizer.save_pretrained("/openbayes/home/my_edited_model")

('/openbayes/home/my_edited_model/tokenizer_config.json',
 '/openbayes/home/my_edited_model/special_tokens_map.json',
 '/openbayes/home/my_edited_model/vocab.json',
 '/openbayes/home/my_edited_model/merges.txt',
 '/openbayes/home/my_edited_model/added_tokens.json')

这会将模型的权重、配置文件、tokenizer 等保存到指定目录。

🔁 如何加载保存后的模型

下次使用时，你可以直接加载这个目录：

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/openbayes/home/my_edited_model").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/openbayes/home/my_edited_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

然后你就可以继续使用这个已经编辑过的模型进行生成、评估或进一步编辑。

In [6]:
prompts = [
    "Question:What sport does Lionel Messi play? Answer: ",
]

# tokenizer = editor.tok
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'left'
# edited_model.config.pad_token_id = tokenizer.pad_token_id

batch = tokenizer(
    prompts,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=30
).to('cuda')

outputs = model.generate(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask'],
    max_new_tokens=10,
    do_sample=False,  # 禁用 sampling
    num_beams=1,      # 禁用 beam search
    pad_token_id=tokenizer.pad_token_id
)

generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for i, text in enumerate(generated_texts):
    print(f"Prompt {i}: {text}")
    print("="*50)

Prompt 0: Question:What sport does Lionel Messi play? Answer: The basketball team at the University of Argentina. How


🧠 进阶建议

如果使用的是 EasyEdit 的 BaseEditor，可以将 edited_model 替换为你加载的模型：

In [ ]:
# editor.model = model
# editor.tok = tokenizer

如果你希望保存多个版本（如不同 subject 的编辑），可以按目录命名区分：
```
/openbayes/home/edited_models/messi_basketball/
/openbayes/home/edited_models/einstein_physicist/
```